In [4]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv(r"C:\Users\LENOVO\Downloads\lab-2.data.csv")

In [8]:
df.columns = df.columns.str.strip()

In [10]:
print("Initial DataFrame head:")
print(df.head())
print("\nInitial DataFrame info:")
print(df.info())

Initial DataFrame head:
  UPDA Unnamed: 1    ENTRY_D EVENT_LCL_DA Unnamed: 4 Unnamed: 5 EVENT_LCL_TI  \
0   No        TED  19-Feb-16          ATE         TE  19-Feb-16           ME   
1   No  19-Feb-16        NaN          NaN  18-Feb-16        NaN    23:55:00Z   
2   No  19-Feb-16        NaN          NaN  18-Feb-16        NaN    22:14:00Z   
3   No  19-Feb-16        NaN          NaN  18-Feb-16        NaN    17:10:00Z   
4   No  19-Feb-16        NaN          NaN  18-Feb-16        NaN    00:26:00Z   

  LOC_CITY_NAME  Unnamed: 8    LOC_STATE_NA  ... CBN_CRW_INJ_NO  \
0     00:45:00Z  MARSHVILLE  North Carolina  ...             NK   
1     TAVERNIER         NaN         Florida  ...            NaN   
2       TRENTON         NaN      New Jersey  ...            NaN   
3     ASHEVILLE         NaN  North Carolina  ...            NaN   
4     TALKEETNA         NaN          Alaska  ...              1   

  CBN_CRW_INJ_MIN CBN_CRW_INJ_SERIO CBN_CRW_INJ_FAT CBN_CRW_INJ_U PAX_INJ_N  \
0            

In [12]:
df['EVENT_LCL_TI'] = df['EVENT_LCL_TI'].str.replace('Z', '', regex=False)

In [14]:
df['DATETIME'] = pd.to_datetime(df['EVENT_LCL_DA'] + ' ' + df['EVENT_LCL_TI'], format='%d-%b-%y %H:%M:%S', errors='coerce')

In [16]:
required_cols = [
    'ACFT_MAKE_NAME', 'LOC_STATE_NA', 'ACFT_MODEL_NA', 'RMK_TEXT',
    'FLT_PHASE', 'EVENT_TYPE_DE', 'FATAL_FL', 'ACFT_DMG_DE'
]
# Create a new DataFrame containing only these columns. Using .copy() prevents SettingWithCopyWarning.
df_clean = df[required_cols].copy()
# Add the newly created 'DATETIME' column to the clean DataFrame.
df_clean['DATETIME'] = df['DATETIME']


In [18]:
# Replace all Fatal Flag missing values with "No"
# Explicitly reassign the column to avoid the FutureWarning.
df_clean['FATAL_FL'] = df_clean['FATAL_FL'].fillna('No')

In [20]:
mode_flt_phase = df_clean['FLT_PHASE'].mode()
if not mode_flt_phase.empty: # Check if mode was found (i.e., column isn't all NaN)
    # Explicitly reassign the column to avoid the FutureWarning.
    df_clean['FLT_PHASE'] = df_clean['FLT_PHASE'].fillna(mode_flt_phase[0])

In [22]:
# Calculate the mode for 'ACFT_DMG_DE'.
mode_acft_dmg_de = df_clean['ACFT_DMG_DE'].mode()
if not mode_acft_dmg_de.empty: # Check if mode was found
    # Explicitly reassign the column to avoid the FutureWarning.
    df_clean['ACFT_DMG_DE'] = df_clean['ACFT_DMG_DE'].fillna(mode_acft_dmg_de[0])

In [24]:
print("\nMissing values after filling:")
print(df_clean.isnull().sum())


Missing values after filling:
ACFT_MAKE_NAME    15
LOC_STATE_NA       4
ACFT_MODEL_NA     71
RMK_TEXT           4
FLT_PHASE          0
EVENT_TYPE_DE      4
FATAL_FL           0
ACFT_DMG_DE        0
DATETIME          87
dtype: int64


In [26]:
# --- Check the number of observations before dropping ---
original_obs_count = len(df_clean)
print(f"\nOriginal number of observations before final dropping: {original_obs_count}")



Original number of observations before final dropping: 87


In [28]:
# Remove all observations where aircraft names ('ACFT_MAKE_NAME') are not available.
# dropna(subset=...) removes rows where NaNs appear only in specified columns.
df_clean.dropna(subset=['ACFT_MAKE_NAME'], inplace=True)

In [30]:
# Drop all columns that have more than 75 non-null values.
# First, count the non-null values for each column in the current (cleaned) DataFrame.
non_null_counts = df_clean.count()
# Identify columns where the non-null count is strictly greater than 75.
cols_to_drop = non_null_counts[non_null_counts > 75].index.tolist()
# Drop these identified columns. errors='ignore' prevents error if a column is not found.
df_clean.drop(columns=cols_to_drop, inplace=True, errors='ignore')

In [32]:
# --- Check the number of observations now to compare it with the original dataset ---
final_obs_count = len(df_clean)
print(f"\nNumber of observations after dropping rows and columns: {final_obs_count}")
print(f"Number of observations dropped from the initial clean DataFrame: {original_obs_count - final_obs_count}")


Number of observations after dropping rows and columns: 72
Number of observations dropped from the initial clean DataFrame: 15


In [34]:
if 'ACFT_MAKE_NAME' in df_clean.columns:
    aircraft_counts = df_clean['ACFT_MAKE_NAME'].value_counts()
    print("\nNumber of times each aircraft type appears in the dataset:")
    print(aircraft_counts)
else:
    print("\n'ACFT_MAKE_NAME' column was dropped due to the specified rule. Cannot group by aircraft type.")



Number of times each aircraft type appears in the dataset:
ACFT_MAKE_NAME
172              4
210              4
150              3
206              3
BOEING           3
PA34             3
PA28             3
8KCAB            2
PA24             2
182              2
310              2
152              2
M20K             2
GULFSTREAM200    1
TB21             1
AIRBUS           1
AA1              1
KITFOX           1
47G              1
CTLS             1
BEECH            1
175              1
PA31             1
IV               1
RX8              1
500              1
M20E             1
NA265            1
AERONCA          1
INTERNATIONAL    1
EMBRAER          1
140              1
2T1A             1
235              1
FIREFLY          1
T34              1
EAGLE II         1
369              1
GC1B             1
PA20             1
560              1
S2               1
BT15             1
PT19             1
TB9              1
SAAB             1
RV7              1
M4               1
M20M         

In [36]:
# --- Display the observations where fatal flag is “Yes” ---
if 'FATAL_FL' in df_clean.columns:
    fatal_incidents = df_clean[df_clean['FATAL_FL'] == 'Yes']
    print("\nObservations where the fatal flag is 'Yes':")
    print(fatal_incidents)
else:
    print("\n'FATAL_FL' column was dropped due to the specified rule. Cannot filter fatal incidents.")



Observations where the fatal flag is 'Yes':
   ACFT_MAKE_NAME    LOC_STATE_NA ACFT_MODEL_NA  \
0           BEECH  North Carolina            ME   
53           PA28         Florida           NaN   
55           CTLS      California           NaN   
79            F51         Arizona           NaN   
80          8KCAB      California           NaN   
82            182         Alabama           NaN   

                                             RMK_TEXT FLT_PHASE EVENT_TYPE_DE  \
0                                                  ME       ITY      Accident   
53  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES. ...   UNKNOWN      Accident   
55  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES A...   UNKNOWN      Accident   
79  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES, ...   UNKNOWN      Accident   
80  N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...   UNKNOWN      Accident   
82  N784CP AIRCRAFT CRASHED INTO A WOODED AREA NEA...   UNKNOWN      Accident   

   FATAL_FL ACFT_DMG_DE DATET

In [38]:

# --- One-hot encoding on the ACFT_DMG_DE column ---
# This is done without using the sklearn library, using pandas' get_dummies.
if 'ACFT_DMG_DE' in df_clean.columns:
    # pd.get_dummies converts categorical variables into dummy/indicator variables.
    one_hot_encoded = pd.get_dummies(df_clean['ACFT_DMG_DE'], prefix='ACFT_DMG_DE')
    
    # Drop one of the resulting columns to avoid multicollinearity (the "dummy variable trap").
    # This means if you have N categories, you only need N-1 binary columns.
    if not one_hot_encoded.empty and len(one_hot_encoded.columns) > 1: # Ensure there's more than one column to drop
        one_hot_encoded.drop(columns=one_hot_encoded.columns[-1], inplace=True)
    
    # Concatenate the new one-hot encoded columns to the main dataframe.
    # First, drop the original 'ACFT_DMG_DE' column to avoid redundancy.
    df_final = pd.concat([df_clean.drop('ACFT_DMG_DE', axis=1), one_hot_encoded], axis=1)

    print("\nFinal DataFrame head after one-hot encoding:")
    print(df_final.head())
    print("\nFinal DataFrame info after one-hot encoding:")
    print(df_final.info())
    
    # Save the final processed dataframe to a CSV file.
    df_final.to_csv('processed_aviation_data.csv', index=False)
    print("\nProcessed data has been saved to 'processed_aviation_data.csv'")
else:
    print("\n'ACFT_DMG_DE' column was dropped due to the specified rule. Cannot perform one-hot encoding.")


Final DataFrame head after one-hot encoding:
  ACFT_MAKE_NAME    LOC_STATE_NA ACFT_MODEL_NA  \
0          BEECH  North Carolina            ME   
1            RV7         Florida           NaN   
2            172      New Jersey           NaN   
3            235  North Carolina           NaN   
4            172          Alaska           NaN   

                                            RMK_TEXT      FLT_PHASE  \
0                                                 ME            ITY   
1  AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...  LANDING (LDG)   
2  AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...       APPROACH   
3  AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...  LANDING (LDG)   
4  AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...  LANDING (LDG)   

  EVENT_TYPE_DE FATAL_FL DATETIME  ACFT_DMG_DE_Destroyed  ACFT_DMG_DE_Minor  \
0      Accident      Yes      NaT                  False              False   
1      Incident       No      NaT                  False            